In [1]:
%cd ..

/Users/yehormishchyriak/Desktop/BonhamLab/summer2025/microbiome2function/data_preparation


In [2]:
import pandas as pd
import data_prep_scripts

In [3]:
df = pd.read_csv(data_prep_scripts.FETCHED_DATA)

In [4]:
new_df = data_prep_scripts.process_entries(df)

No preprocessing rule for 'Entry' — leaving as is.


# Before pre-processing

In [5]:
cols = df.columns

summary = []
for col in cols:
    non_null = df[col].count()
    unique   = df[col].nunique(dropna=True)
    duplication_rate = (non_null - unique) / non_null if non_null else float("nan")
    summary.append({
        "column": col,
        "dimensionality": unique,
        "% entries sharing it": f"{duplication_rate:.2%}",
        "% non-null": f"{non_null / len(df):.2%}"
    })

summary_df = pd.DataFrame(summary)
print(summary_df)

                                column  dimensionality % entries sharing it  \
0                                Entry             263                0.00%   
1                          Domain [FT]           26047                8.78%   
2                          Domain [CC]             328               50.15%   
3                     Protein families            2714               87.84%   
4   Gene Ontology (molecular function)            6310               83.34%   
5   Gene Ontology (biological process)            2786               89.00%   
6                       Interacts with             367                8.93%   
7                        Function [CC]            4594               56.18%   
8                   Catalytic activity            3273               72.90%   
9                            EC number            2580               89.95%   
10                             Pathway            1529               72.22%   
11                             Rhea ID            14

# After pre-processing

In [6]:
cols = new_df.columns

summary = []
for col in cols:
    non_null = new_df[col].count()
    unique   = new_df[col].nunique(dropna=True)
    duplication_rate = (non_null - unique) / non_null if non_null else float("nan")
    summary.append({
        "column": col,
        "dimensionality": unique,
        "% entries sharing it": f"{duplication_rate:.2%}",
        "% non-null": f"{non_null / len(new_df):.2%}"
    })

summary_new_df = pd.DataFrame(summary)
print(summary_new_df)

                                column  dimensionality % entries sharing it  \
0                                Entry             263                0.00%   
1                          Domain [FT]            3388               88.13%   
2                          Domain [CC]             273               58.51%   
3                     Protein families            2686               87.97%   
4   Gene Ontology (molecular function)            6310               83.34%   
5   Gene Ontology (biological process)            2786               89.00%   
6                       Interacts with             367                8.93%   
7                        Function [CC]            3393               67.64%   
8                   Catalytic activity            1583               86.89%   
9                            EC number            2580               89.95%   
10                             Pathway             644               88.30%   
11                             Rhea ID            14